<h1>
    Capstone: Bop It Game
</h1>

In [13]:
import RPi.GPIO as gp
import numpy as np
import accUtil as acc
import adcUtil as adc
from time import sleep, time
import random


#variables

BUTTON_PIN = 23 #pin number for the button
GATE_PIN = 22 #pin number for the mic gate
MIC_CHANNEL = 0 #ADC channel for the mic
POT_CHANNEL = 1 #ADC channel for the potentiometer
POT_THRESHOLD = 1.65 #determines how far to either side the potentiometer has to be turned to register a change
LED_PINS = {"mic": 27, "twist": 26, "shake": 25, "button": 24} #dictionary of LED pin numbers, with each LED corresponding to
                                                               #a different sensor
micData = np.array([]) #mic gate data array
potData = np.array([]) #potentiometer data array
accelData = np.array([]) #accelerometer data array
buttonData = np.array([]) #button data array
timeData = np.array([]) #timestamps for reference against the other data arrays

#classes
class game:
    
    #TODO: Currently treating the LED_PINS data as a list instead of a dict, either need to change original data
    #variable or implementation in the class
    
    def __init__(self, LED_PINS):
        self.gameStart = time.time() # Time when the game object is created (when the button is pressed)
        self.reactionTime = 8 # Time for the player to do the action, will go down over span of game
        self.LED_PINS = LED_PINS 
        self.points = 0 # Game points the player has earned
        self.lostGame = False
        # Flash the LEDs 3 times to indicate the game started
        for i in range(3):
            for pin in self.LED_PINS:
                gp.output(pin, True)
            time.sleep(0.2)
            for pin in self.LED_PINS:
                gp.output(pin, False)
        time.sleep(2) # Give the player 2 seconds to prepare
        
        
    def runGame(self):
        # Run the game until the player loses
        while self.lostGame == False:
            choice = chooseAction()
            waitingStartTime = time.time() # Start the timer for the player to react
            while (time.time() - waitingStartTime) <= self.reactionTime:
                '''# TODO: Get all the sensor data into an array as booleans, then check if the action 
                # we expect has been performed
                #data = {microphone_data, potChange(), accelerometer_data, gp.input(BUTTON_PIN)}
                # May need to get sensor data into the class
                
                # If the action has been performed, increment the points, decrement the reactionTime (to a limit,
                # can't give the player a reaction time of zero. Thinking minimum 2 seconds.) 
                # Maybe do this all in a separate function
                    
                    
        # TODO: If action is not performed in the time permitted (once the loop exits), set lostGame to true and run
        # gameOver()'''
        
        
        # Chooses which action and flashes the LED of which action the player needs to take
        # 0 is microphone, 1 is potentiometer, 2 is accelerometer, 3 is button
    def chooseAction(self):
        choice = random.randint(0,3) # Generate a random number to choose which player Action
        #Flash the LED based on which action was chosen
        gp.output(self.LED_PINS[choice], True)
        time.sleep(1)
        gp.output(self.LED_PINS[choice], False)
        return choice
    
    def gameOver(self):
        # Flash all LEDs 3 times quickly to indicate the game has ended
        for i in range(3):
            for pin in self.LED_PINS:
                gp.output(pin, True)
            time.sleep(0.1)
            for pin in self.LED_PINS:
                gp.output(pin, False)
                
        time.sleep(1) # Wait a second to give the player pause on how they just lost
        
        # Flash the LEDs for the number of points the player got
        for i in range(self.points):
            for pin in self.LED_PINS:
                gp.output(pin, True)
            time.sleep(0.2)
            for pin in self.LED_PINS:
                gp.output(pin, False)
#methods

def setup():
    '''Setup the gpio pins.'''
    
    gp.setmode(gp.BCM)
    
    for pin in LED_PINS:
        gp.setup(LED_PINS[pin], gp.OUT)
        
    gp.setup(BUTTON_PIN, gp.IN, gp.PUD_DOWN)
    gp.setup(GATE_PIN, gp.IN)
    
def potChange():
    '''Detect whether or not the potentiometer has been turned enough to register a change.
    Returns:
        True if a change was registered, false otherwise.'''
    
    if potData.size <= 2:
        return False
    
    #check if potentiometer voltage went from one side of the threshold to another since last measurement
    if potData[potData.size - 1] >= POT_THRESHOLD:
        return potData[potData.size - 2] < POT_THRESHOLD
    else:
        return potData[potData.size - 2] >= POT_THRESHOLD
    
    
#main

try:
    
    setup() #setup gpio
    
    #game loop
    while True:
        if (gp.input(BUTTON_PIN)):
            game = game(LED_PINS)
            
        #record data
        micData = np.append(micData, gp.input(GATE_PIN))
        potData = np.append(potData, adc.readADC(POT_CHANNEL))
        accelData = np.append(accelData, acc.readACC())
        buttonData = np.append(buttonData, gp.input(BUTTON_PIN))
        timeData = np.append(timeData, time())
    
except: #KeyboardInterrupt
    print("An exception occured.")
finally:
    gp.cleanup()

Change detected
Change detected
Change detected
Change detected
Change detected
Change detected
Change detected
Change detected
Change detected
Change detected
An exception occured.
